<a href="https://colab.research.google.com/github/rossel/DataQuest_Courses/blob/master/020__Data_Cleaning_Basics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MISSION 6: Data Cleaning Basics

In this mission, we will learn techniques to use when performing data cleaning to prepare a messy data set,by following the steps in the workflow diagram below:

![alt text](https://s3.amazonaws.com/dq-content/293/cleaning_workflow.svg)

## 1. Reading CSV Files with Encodings

In this mission, we'll learn the basics of data cleaning with pandas as we work with `laptops.csv`, a CSV file containing information about 1,300 laptop computers.

We can start by reading the data into pandas. Let's look at what happens when we use the `pandas.read_csv()` [function](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html) with only the filename argument:


```
laptops = pd.read_csv("laptops.csv")
```

We get an error! This error references UTF-8, which is a type of encoding. Computers, at their lowest levels, can only understand binary - `0` and `1` - and encodings are systems for representing characters in binary.

Something we can do if our file has an unknown encoding is to **try the most common encodings**:

* UTF-8
* Latin-1 (also known as ISO-8859-1)
* Windows-1251

The `pandas.read_csv()` function has an `encoding` argument we can use to specify an encoding:



```
df = pd.read_csv("filename.csv", encoding="some_encoding")
```
Since the `pandas.read_csv()` function already tried to read in the file with UTF-8 and failed, we know the file's not encoded with that format. Let's try the next most popular encoding in the exercise.



First, let's import `laptops.csv` into this Colaboratory Notebook:

In [22]:
# Code to read csv file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [23]:
# Once you have completed verification, go to the CSV file in Google Drive, right-click on it and select “Get shareable link”, and cut out the unique id in the link.
id = "1VjDliDrTkzvTvjz6uRnjt7Z72uv6TI6s"

In [24]:
# Download the dataset
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('laptops.csv')

Instructions:


1. Import the pandas library
2. Use the `pandas.read_csv()` function to read the `laptops.csv`file into a dataframe `laptops`.
  * Specify the encoding using the string `"Latin-1"`.
3. Use the `DataFrame.info()` method to display information about the `laptops` dataframe.


In [25]:
import pandas as pd
laptops = pd.read_csv("laptops.csv", encoding="Latin-1")

laptops.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1303 entries, 0 to 1302
Data columns (total 13 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Manufacturer              1303 non-null   object
 1   Model Name                1303 non-null   object
 2   Category                  1303 non-null   object
 3   Screen Size               1303 non-null   object
 4   Screen                    1303 non-null   object
 5   CPU                       1303 non-null   object
 6   RAM                       1303 non-null   object
 7    Storage                  1303 non-null   object
 8   GPU                       1303 non-null   object
 9   Operating System          1303 non-null   object
 10  Operating System Version  1133 non-null   object
 11  Weight                    1303 non-null   object
 12  Price (Euros)             1303 non-null   object
dtypes: object(13)
memory usage: 132.5+ KB


In [26]:
print(laptops.shape)

(1303, 13)


## 2. Cleaning Column Names

With the `DataFrame.info()` used above, we can see that every column is represented as the `object` type, indicating that they are represented by strings, not numbers. Also, one of the columns, `Operating System Version`, has null values. 

The column labels have a variety of **upper and lowercase letters**, as well as **spaces and parentheses**, which will make them harder to work with and read. One noticeable issue is that the ` Storage"` column name has a space in front of it. These quirks with column labels can sometimes be hard to spot, so removing extra whitespaces from all column names will save us more work in the long run.

We can access the column axis of a dataframe using the `DataFrame.columns` attribute. This returns an index object — a special type of NumPy ndarray — with the labels of each column:

In [27]:
print(laptops.columns)

Index(['Manufacturer', 'Model Name', 'Category', 'Screen Size', 'Screen',
       'CPU', 'RAM', ' Storage', 'GPU', 'Operating System',
       'Operating System Version', 'Weight', 'Price (Euros)'],
      dtype='object')


Not only can we use the attribute to view the column labels, we can also assign new labels to the attribute:



```
laptops_test = laptops.copy()
laptops_test.columns = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M']

print(laptops_test.columns)
```


```
Index(['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M'], dtype='object')
```
Next, let's use the `DataFrame.columns` attribute to remove whitespaces from the column names.



In [28]:
new_columns = []

for i in laptops.columns:
    stripped = i.strip(" ")
    new_columns.append(stripped)
    
laptops.columns = new_columns

print(laptops.columns)

Index(['Manufacturer', 'Model Name', 'Category', 'Screen Size', 'Screen',
       'CPU', 'RAM', 'Storage', 'GPU', 'Operating System',
       'Operating System Version', 'Weight', 'Price (Euros)'],
      dtype='object')


## 3. Cleaning Column Names Continued

he column labels still have a variety of upper and lowercase letters, as well as parentheses, which will make them harder to work with and read. Let's finish cleaning our column labels by:

* Replacing spaces with underscores.
* Removing special characters.
* Making all labels lowercase.
* Shortening any long column names.

We can create a function that uses [Python string methods](https://docs.python.org/3/library/stdtypes.html#string-methods) to clean our column labels, and then again use a loop to apply that function to each label. Let's look at an example:



```
def clean_col(col):
    col = col.strip()
    col = col.replace("(","")
    col = col.replace(")","")
    col = col.lower()
    return col

new_columns = []
for c in laptops.columns:
    clean_c = clean_col(c)
    new_columns.append(clean_c)

laptops.columns = new_columns
print(laptops.columns)
```



```
Index(['manufacturer', 'model name', 'category', 'screen size', 'screen',
       'cpu', 'ram', 'storage', 'gpu', 'operating system',
       'operating system version', 'weight', 'price euros'],
      dtype='object')
```



Our code:

* Defined a function, which:
  * Used the `str.strip()` [method](https://docs.python.org/3.6/library/stdtypes.html#str.strip) to remove whitespace from the start and end of the string.
  * Used the `str.replace()` [method](https://docs.python.org/3.6/library/stdtypes.html#str.replace) to remove parentheses from the string.
  * Used the `str.lower()` [method](https://docs.python.org/3.6/library/stdtypes.html#str.lower) to make the string lowercase.
  * Returns the modified string.
* Used a loop to apply the function to each item in the index object and assign it back to the `DataFrame.columns` attribute.
* Printed the new values for the `DataFrame.columns` attribute.

Let's use this technique to clean the column labels in our dataframe, adding a few extra cleaning 'chores' along the way.


1. Define a function, which accepts a string argument, and:
  * Removes any whitespace from the start and end of the string.
  * Replaces the substring `Operating System` with the abbreviation `os`.
  * Replaces all spaces with underscores.
  * Removes parentheses from the string.
  * Makes the entire string lowercase.
  * Returns the modified string.
2. Use a loop to apply the function to each item in the `DataFrame.columns` attribute for the `laptops` dataframe. Assign the result back to the `DataFrame.columns` attribute.


In [29]:
def clean_col(col):
    col = col.strip()
    col = col.replace("Operating System","os")
    col = col.replace(" ","_")
    col = col.replace("(","")
    col = col.replace(")","")
    col = col.lower()
    return col

columns_clean = []

for i in laptops.columns:
    col = clean_col(i)
    columns_clean.append(col)
    
laptops.columns = columns_clean

print(laptops.columns)

Index(['manufacturer', 'model_name', 'category', 'screen_size', 'screen',
       'cpu', 'ram', 'storage', 'gpu', 'os', 'os_version', 'weight',
       'price_euros'],
      dtype='object')


## 4. Converting String Columns to Numeric

We observed earlier that all 13 columns have the `object` dtype, meaning they're stored as strings. Let's look at the first few rows of some of our columns:

In [30]:
print(laptops.iloc[:5,2:5])

    category screen_size                              screen
0  Ultrabook       13.3"  IPS Panel Retina Display 2560x1600
1  Ultrabook       13.3"                            1440x900
2   Notebook       15.6"                   Full HD 1920x1080
3  Ultrabook       15.4"  IPS Panel Retina Display 2880x1800
4  Ultrabook       13.3"  IPS Panel Retina Display 2560x1600


Of these three columns, we have three different types of text data:

* `category`: Purely text data - there are no numeric values.
* `screen_size`: Numeric data stored as text data because of the `"` character.
* `screen`: A combination of pure text data with numeric data.

Because the values in the `screen_size` column are stored as text data, we can't sort them numerically. For instance, if we wanted to select laptops with screens 15" or larger, we'd be unable to do so.

Let's convert the `screen_size` column to numeric next. Whenever we convert text to numeric data, we can follow this data cleaning workflow diagram:

![alt text](https://s3.amazonaws.com/dq-content/293/cleaning_workflow.svg)


The first step is to **explore the data**. One of the best ways to do this is to use the `Series.unique()` method to view all of the unique values in the column:

In [31]:
print(laptops["screen_size"].dtype)
print(laptops["screen_size"].unique())

object
['13.3"' '15.6"' '15.4"' '14.0"' '12.0"' '11.6"' '17.3"' '10.1"' '13.5"'
 '12.5"' '13.0"' '18.4"' '13.9"' '12.3"' '17.0"' '15.0"' '14.1"' '11.3"']




Our next step is to **identify patterns and special cases**. We can observe the following:

* All values in this column follow the same pattern - a series of digit and period characters, followed by a quote character (`"`).
* There are no special cases. Every value matches the same pattern.
* We'll need to convert the column to a `float` dtype, as the `int` dtype won't be able to store the decimal values.

Let's identify any patterns and special cases in the `ram` column next.

In [32]:
unique_ram = laptops["ram"].unique()

print(unique_ram)

['8GB' '16GB' '4GB' '2GB' '12GB' '6GB' '32GB' '24GB' '64GB']


Here, we can identify a clear pattern in the `ram` column - all values are integers and include the character `GB` at the end of the string.



## 5. Removing Non-Digit Characters

To convert both the `ram` and `screen_size` columns to numeric dtypes, we'll have to first remove the non-digit characters (As depicted in the flow-chart above).

The pandas library contains dozens of [vectorized string methods](https://pandas.pydata.org/pandas-docs/stable/user_guide/text.html#method-summary) we can use to manipulate text data, many of which perform the same operations as Python string methods. Most vectorized string methods are available using the `Series.str` [accessor](http://pandas.pydata.org/pandas-docs/stable/api.html#string-handling), which means we can access them by adding `str` between the series name and the method name:

![alt text](https://s3.amazonaws.com/dq-content/346/Syntax.png)



In this case, we can use the `Series.str.replace()` method, which is a vectorized version of the Python `str.replace()` method we used in the previous screen, to remove all the quote characters from every string in the `screen_size` column:

In [33]:
laptops["screen_size"] = laptops["screen_size"].str.replace('"','')

print(laptops["screen_size"].unique())

['13.3' '15.6' '15.4' '14.0' '12.0' '11.6' '17.3' '10.1' '13.5' '12.5'
 '13.0' '18.4' '13.9' '12.3' '17.0' '15.0' '14.1' '11.3']


Let's remove the non-digit characters from the ram column next.

In [34]:
laptops["ram"] = laptops["ram"].str.replace("GB","")
print(laptops["ram"].unique())

['8' '16' '4' '2' '12' '6' '32' '24' '64']


## 6. Converting Columns to Numeric Dtypes

Now, we can **convert (or cast) the columns to a numeric dtype which is Step 5 in the flow chart image**.

![alt text](https://s3.amazonaws.com/dq-content/293/cleaning_workflow.svg)

To do this, we use the `Series.astype()` [method](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.astype.html). To convert the column to a numeric dtype, we can use either `int` or `float` as the parameter for the method. Since the `int` dtype can't store decimal values, we'll convert the `screen_size` column to the `float` dtype:

In [35]:
laptops["screen_size"] = laptops["screen_size"].astype(float)
print(laptops["screen_size"].dtype)
print(laptops["screen_size"].unique())

float64
[13.3 15.6 15.4 14.  12.  11.6 17.3 10.1 13.5 12.5 13.  18.4 13.9 12.3
 17.  15.  14.1 11.3]


Our `screen_size` column is now the `float64` dtype. Let's convert the dtype of the `ram` column to numeric next.

In [36]:
laptops["ram"] = laptops["ram"].astype(int)
laptops.dtypes

manufacturer     object
model_name       object
category         object
screen_size     float64
screen           object
cpu              object
ram               int64
storage          object
gpu              object
os               object
os_version       object
weight           object
price_euros      object
dtype: object

## 7. Renaming Columns

Now that we've converted our columns to numeric dtypes, the **final step is to rename the column**. 

This is the **last step** in the work flow diagram above, and an **optional** step, which can be **useful** if the non-digit values contain information that helps us understand the data.

In our case, the quote characters we removed from the `screen_size` column denoted that the screen size was in inches.



To stop us from losing information the helps us understand the data, we can use the `DataFrame.rename()` [method](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.rename.html) to rename the column from `screen_size` to `screen_size_inches`.

Below, we specify the `axis=1` parameter so pandas knows that we want to rename labels in the column axis:



In [37]:
laptops.rename({"screen_size": "screen_size_inches"}, axis=1, inplace=True)

print(laptops.dtypes)

manufacturer           object
model_name             object
category               object
screen_size_inches    float64
screen                 object
cpu                    object
ram                     int64
storage                object
gpu                    object
os                     object
os_version             object
weight                 object
price_euros            object
dtype: object


Note that we can either use `inplace=True` or assign the result back to the dataframe - both will give us the same results.

Let's rename the `ram` column next and analyze the results.

In [38]:
laptops.rename({"ram": "ram_gb"}, axis=1, inplace=True)

In [39]:
ram_gb_desc = laptops["ram_gb"].describe()

print(ram_gb_desc)

count    1303.000000
mean        8.382195
std         5.084665
min         2.000000
25%         4.000000
50%         8.000000
75%         8.000000
max        64.000000
Name: ram_gb, dtype: float64


## 8. Extracting Values from Strings

Sometimes, it can be useful to extract non-numeric values from within strings. Let's look at the first five values from the `gpu` (graphics processing unit) column:



In [40]:
print(laptops["gpu"].head())

0    Intel Iris Plus Graphics 640
1          Intel HD Graphics 6000
2           Intel HD Graphics 620
3              AMD Radeon Pro 455
4    Intel Iris Plus Graphics 650
Name: gpu, dtype: object


The information in this column seems to be a manufacturer (Intel, AMD) followed by a model name/number. Let's extract the manufacturer by itself so we can find the most common ones.

Because each manufacturer is followed by a whitespace character, we can use the `Series.str.split()` [method](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.str.split.html) to extract this data:

![alt text](https://s3.amazonaws.com/dq-content/293/str_split_2.svg)

This method splits each string on the whitespace; the result is a series containing individual Python lists. Also note that we used parentheses to method chain over multiple lines, which makes our code easier to read.

Just like with lists and ndarrays, we can use bracket notation to access the elements in each list in the series. With series, however, we use the `str` accessor followed by `[]` (brackets):



In [41]:
print(laptops["gpu"].head().str.split().str[0])

0    Intel
1    Intel
2    Intel
3      AMD
4    Intel
Name: gpu, dtype: object


Above, we used `0` to select the first element in each list. 



Now let's extract the manufacturer name from the `gpu` column, and assigned it to a new column `gpu_manufacturer`:

In [43]:
laptops["gpu_manufacturer"] = (laptops["gpu"]
                                       .str.split()
                                       .str[0]
                              )

Let's use this technique to extract the manufacturer from the `cpu` column as well. Here are the first 5 rows of the `cpu` column:

In [42]:
print(laptops["cpu"].head())

0          Intel Core i5 2.3GHz
1          Intel Core i5 1.8GHz
2    Intel Core i5 7200U 2.5GHz
3          Intel Core i7 2.7GHz
4          Intel Core i5 3.1GHz
Name: cpu, dtype: object


**Instructions:**

In the example code, we have extracted the manufacturer name from the `gpu` column, and assigned it to a new column `gpu_manufacturer`.

1. Extract the manufacturer name from the `cpu` column. Assign it to a new column `cpu_manufacturer`.

In [44]:
laptops["cpu_manufacturer"] = (laptops["cpu"]
                                        .str.split()
                                        .str[0]
                              )

2. Use the `Series.value_counts()` method to find the counts of each manufacturer in `cpu_manufacturer`. Assign the result to `cpu_manufacturer_counts`.

In [45]:
cpu_manufacturer_counts = laptops["cpu_manufacturer"].value_counts()

print(cpu_manufacturer_counts)

Intel      1240
AMD          62
Samsung       1
Name: cpu_manufacturer, dtype: int64


## 9. Correcting Bad Values

If your data has been scraped from a webpage or if there was manual data entry involved at some point, you may end up with inconsistent values. Let's look at an example from our `os` column:

In [46]:
print(laptops["os"].value_counts())

Windows      1125
No OS          66
Linux          62
Chrome OS      27
macOS          13
Mac OS          8
Android         2
Name: os, dtype: int64


We can see that there are two variations of the Apple operating system — macOS — in our data set: `Mac OS` and `macOS`. One way we can fix this is with the `Series.map()` method. The `Series.map()` method is ideal when we want to change multiple values in a column, but we'll use it now as an opportunity to learn how the method works.

The most common way to use `Series.map()` is with a dictionary. Let's look at an example using a series of misspelled fruit:

```
print(s)
```


```
0       pair
1     oranje
2    bananna
3     oranje
4     oranje
5     oranje
dtype: object
```
We'll create a dictionary called `corrections` and pass that dictionary as an argument to `Series.map()`:


```
corrections = {
    "pair": "pear",
    "oranje": "orange",
    "bananna": "banana"
}
s = s.map(corrections)
print(s)
```


```
0       pear
1     orange
2     banana
3     orange
4     orange
5     orange
dtype: object
```

We can see that each of our corrections were made across our series. One important thing to remember with `Series.map()` is that if a value from your series doesn't exist as a key in your dictionary, it will convert that value to `NaN`.

Let's use Series.map() to clean the values in the os column.

In [48]:
mapping_dict = {
    'Android': 'Android',
    'Chrome OS': 'Chrome OS',
    'Linux': 'Linux',
    'Mac OS': 'macOS',
    'No OS': 'No OS',
    'Windows': 'Windows',
    'macOS': 'macOS'
}

laptops["os"] = laptops["os"].map(mapping_dict)

print(laptops["os"].head())

0    macOS
1    macOS
2    No OS
3    macOS
4    macOS
Name: os, dtype: object


## 10. Dropping Missing Values

In previous missions, we've talked briefly about missing values and how both NumPy and pandas represent these as null values. In pandas, null values will be indicated by either `NaN` or `None`.

Recall that we can use the `DataFrame.isnull()` [method](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.isnull.html) to identify missing values, which returns a boolean dataframe. We can then use the `DataFrame.sum()` method to give us a count of the `True` values for each column:



In [49]:
print(laptops.isnull().sum())

manufacturer            0
model_name              0
category                0
screen_size_inches      0
screen                  0
cpu                     0
ram_gb                  0
storage                 0
gpu                     0
os                      0
os_version            170
weight                  0
price_euros             0
gpu_manufacturer        0
cpu_manufacturer        0
dtype: int64


It's now clear that we have only one column with null values, `os_version`, which has 170 missing values.

There are a few options for handling missing values:

* Remove any rows that have missing values.
* Remove any columns that have missing values.
* Fill the missing values with some other value.
* Leave the missing values as is.

The first two options are often used to prepare data for machine learning algorithms, which are unable to be used with data that includes null values. We can use the `DataFrame.dropna()` [method](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.dropna.html) to remove or drop rows and columns with null values.

The `DataFrame.dropna()` method accepts an `axis` parameter, which indicates whether we want to drop along the column or index axis. Let's look at an example:

![alt text](https://s3.amazonaws.com/dq-content/293/dropna_1.svg)

The default value for the `axis` parameter is `0`, so `df.dropna()` returns an identical result to `df.dropna(axis=0)`:

![alt text](https://s3.amazonaws.com/dq-content/293/dropna_2.svg)

The rows with labels `x` and `z` contain null values, so those rows are dropped. Let's look at what happens when we use `axis=1` to specify the column axis:
![alt text](https://s3.amazonaws.com/dq-content/293/dropna_3.svg)

Only the column with label `C` contains null values, so, in this case, just one column is removed.

Let's practice using `DataFrame.dropna()` to remove rows and columns:

1. Use DataFrame.dropna() to remove any rows from the laptops dataframe that have null values. Assign the result to laptops_no_null_rows.
2. Use DataFrame.dropna() to remove any columns from the laptops dataframe that have null values. Assign the result to laptops_no_null_cols.

In [50]:
laptops_no_null_rows = laptops.dropna(axis=0)

laptops_no_null_cols = laptops.dropna(axis=1)

## 11. Filling Missing Values

While dropping rows or columns is the easiest approach to deal with missing values, it may not always be the *best* approach. For example, removing a disproportionate amount of one manufacturer's laptops could change our analysis.

Because of this, it's a good idea to explore the missing values in the `os_version` column before making a decision. We can use `Series.value_counts()` to explore all of the values in the column, but we'll use a parameter we haven't seen before:



In [51]:
print(laptops["os_version"].value_counts(dropna=False))

10      1072
NaN      170
7         45
10 S       8
X          8
Name: os_version, dtype: int64


Because we set the `dropna` parameter to `False`, the result includes null values. We can see that the majority of values in the column are `10` and missing values are the next most common.

Let's also explore the `os` column, since it's is closely related to the `os_version` column. We'll only look at rows in which the `os_version` is missing:



In [52]:
os_with_null_v = laptops.loc[laptops["os_version"].isnull(),"os"]

print(os_with_null_v.value_counts())

No OS        66
Linux        62
Chrome OS    27
macOS        13
Android       2
Name: os, dtype: int64


Immediately, we can observe a few things:

* The most frequent value is "No OS". This is important to note because if there is no os, there shouldn't be a version defined in the `os_version` column.
* Thirteen of the laptops that come with macOS do not specify the version. We can use our knowledge of MacOS to confirm that `os_version` should be equal to X.

In both of these cases, we can fill the missing values to make our data more correct. For the rest of the values, it's probably best to leave them as missing so we don't remove important values.

We can use assignment with a boolean comparison to perform this replacement, like below:

In [53]:
laptops.loc[laptops["os"] == "macOS", "os_version"] = "X"

For rows with `No OS` values, let's replace the missing value in the `os_version` column with the value `Version Unknown`.

Instructions:

Use a boolean array to identify rows that have the value No OS for the os column. Then, use assignment to assign the value Version Unknown to the os_version column for those rows.


In [54]:
laptops.loc[laptops["os"] == "No OS", "os_version"] = "Version Unknown"

## 12. Challenge: Clean a String Column

In this challenge, we'll clean the `weight` column. Let's look at a sample of the data in that column:

In [55]:
print(laptops["weight"].head())

0    1.37kg
1    1.34kg
2    1.86kg
3    1.83kg
4    1.37kg
Name: weight, dtype: object


Your challenge is to convert the values in this column to numeric values. As a reminder, here's the data cleaning workflow you can use:
![alt text](https://s3.amazonaws.com/dq-content/293/cleaning_workflow.svg)

While it appears that the `weight` column may just need the `kg` characters removed from the end of each string, there is one special case - one of the values ends with `kgs`, so you'll have to remove both `kg` and `kgs` characters.

In the last step of this challenge, we'll also ask you to use the `DataFrame.to_csv()` [method](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.to_csv.html) to save the cleaned data to a CSV file. It's a good idea to save a CSV when you finish cleaning in case you wish to do analysis later.

We can use the following syntax to save a CSV:

```
df.to_csv('filename.csv', index=False)
```
By default, pandas will save the index labels as a column in the CSV file. Our data set has integer labels that don't contain any data, so we don't need to save the index.



Instructions:

1. Convert the values in the `weight` column to numeric values.
2. Rename the `weight` column to `weight_kg`.
3. Use the `DataFrame.to_csv()` method to save the laptops dataframe to a CSV file `laptops_cleaned.csv` without index labels.

In [56]:
laptops["weight"] = laptops["weight"].str.replace("kgs","").str.replace("kg","").astype(float)

laptops.rename({"weight": "weight_kg"}, axis=1, inplace=True)

laptops.to_csv('laptops_cleaned.csv',index=False)

Note: a for-loop is not necessary, and the .astype(float) is added to the end. Here is the code again:
```
laptops["weight"] = 
laptops["weight"].str.replace("kgs","")
                 .str.replace("kg","")
                 .astype(float)

```
